In [ ]:
!pip install transformers
!pip install torch
!pip install newspaper3k

In [ ]:
import nltk
nltk.download('punkt')
from textblob import TextBlob
from newspaper import Article

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class Mood:
    emoji: str
    sentiment: float

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")


In [6]:
def sentiment_analysis(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    labels = ['negative', 'neutral', 'positive']
    result = {labels[i]: scores[i] for i in range(len(labels))}
    return result


In [13]:
def get_mood(input_text: str, *, threshold: float) -> Mood:
    sentiment: float = TextBlob(input_text).sentiment.polarity


    friendly_treshold: float = threshold
    hostile_treshold: float = -threshold

    if sentiment > friendly_treshold:
        return Mood("😊", sentiment)
    elif sentiment < hostile_treshold:
        return Mood("😠", sentiment)
    else:
        return Mood("😐", sentiment)



url = "https://www.ensonhaber.com/kralspor/avrupa-sampiyonasi/arda-guler-gol-atti-futbol-dunyasi-kayitsiz-kalmadi"
article = Article(url)
article.download()
article.parse()
article.nlp()

text = article.summary
print(text)

blob = TextBlob(text)
sentiment = blob.sentiment.polarity # 0 to 1
print(sentiment)

mood = get_mood(text, threshold=0.1)
print(mood)

Arda Güler gol attı, futbol dünyası kayıtsız kalmadı A Milli Futbol Takımızın, Gürcistan'ı 3-1'lik skorla yendiği EURO 2024 maçında attığı golle karşılaşmaya damgasını vuran Arda Güler, futbol dünyasının takdirini topladı.
ensonhaber.com2024 Avrupa Futbol Şampiyonası F Grubu ilk maçında A Milli Futbol Takımı BVB Dortmund Stadyumu’nda Gürcistan ile oynadı.
Öte yandan Arda Güler, Cristiano Ronaldo'nun geçerek Avrupa Futbol Şampiyonası tarihinde oynadığı ilk maçında gol atan en genç oyuncu oldu.
İspanyol devi, maçın sonucunu Arda Güler'in gol sevincine ilişkin fotoğraf ile paylaştı.
Mesut ÖzilFenerbahçe ve Real Madrid'in eski yıldızlarından Mesut Özil de Arda Güler için paylaşımda bulundu.
0.2
Mood(emoji='😊', sentiment=0.2)
